# Data Fetching: FJC and Congress.gov API

This notebook is responsible for fetching and initially processing data from our primary sources:

1. Federal Judicial Center (FJC) CSV and Excel files
2. Congress.gov API judicial nomination data

According to the project architecture, this notebook will:
1. Download or use cached data from the FJC and Congress.gov API
2. Perform minimal transformations to convert to dataframes
3. Save the resulting dataframes to `data/raw` for further processing by downstream notebooks

## Setup

In [ ]:
import os
import sys
from pathlib import Path

import pandas as pd
from loguru import logger

# Add the project root to the path so we can import our modules
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from nomination_predictor.congress_api import CongressAPIClient

# Setup logging
logger.remove()  # Remove default handler
logger.add(sys.stderr, format="<green>{time:YYYY-MM-DD HH:mm:ss}</green> | <level>{level}</level> | <cyan>{function}</cyan> - <level>{message}</level>", level="INFO")

7

## 1. Federal Judicial Center (FJC) Data

The FJC data is our canonical source for judicial seat timelines, judge demographics, and nomination failures.

### Check if FJC data exists or download if needed

In [ ]:
# Check if required FJC data files exist and download any missing ones
from nomination_predictor.config import EXTERNAL_DATA_DIR
from nomination_predictor.fjc_data import (REQUIRED_FJC_FILES,
                                           ensure_fjc_data_files,
                                           load_fjc_data)

# Check for missing files and download them if needed
downloaded, failed = ensure_fjc_data_files()

# Report status
if downloaded:
    print(f"✓ Downloaded {len(downloaded)} previously missing files: {', '.join(downloaded)}")
if failed:
    print(f"❌ Failed to download {len(failed)} files: {', '.join(failed)}")
    
# Also report on which files are present
present_files = [f for f in REQUIRED_FJC_FILES if (EXTERNAL_DATA_DIR / f).exists()]
if len(present_files) == len(REQUIRED_FJC_FILES):
    print(f"✓ All required FJC data files are available in {EXTERNAL_DATA_DIR}")
else:
    missing = set(REQUIRED_FJC_FILES) - set(present_files)
    print(f"⚠️ Still missing {len(missing)} required files: {', '.join(missing)}")

2025-07-12 18:24:41 | INFO | ensure_fjc_data_files - Ensuring FJC data files are available


✓ All required FJC data files are available in /home/wsl2ubuntuuser/nomination_predictor/data/external


### Load FJC Data

In [ ]:
# Load all FJC data files (with auto-download enabled by default)
fjc_data = load_fjc_data()

# Access individual DataFrames
print(f"Loaded FJC data files:")
for key, df in fjc_data.items():
    print(f"- {key}: {len(df)} records")

# Store references to commonly used DataFrames for easier access
judges_df = fjc_data.get('judges')
demographics_df = fjc_data.get('demographics')
education_df = fjc_data.get('education')
federal_judicial_service_df = fjc_data.get('federal_judicial_service')
other_nominations_recess_df = fjc_data.get('other_nominations_recess')
other_federal_judicial_service_df = fjc_data.get('other_federal_judicial_service')
professional_career_df = fjc_data.get('professional_career')

# Create a dictionary of all FJC dataframes for easy iteration
all_dataframes = {
    'judges': judges_df,
    'demographics': demographics_df,
    'education': education_df,
    'federal_judicial_service': federal_judicial_service_df,
    'other_nominations_recess': other_nominations_recess_df,
    'other_federal_judicial_service': other_federal_judicial_service_df,
    'professional_career': professional_career_df
}

2025-07-12 18:24:41 | INFO | load_fjc_data - Loading FJC data files
2025-07-12 18:24:41 | INFO | ensure_fjc_data_files - Ensuring FJC data files are available
2025-07-12 18:24:41 | INFO | load_fjc_csv - Loading FJC data file: demographics.csv
2025-07-12 18:24:41 | INFO | load_fjc_data - Loaded demographics data with 4022 records
2025-07-12 18:24:41 | INFO | load_fjc_csv - Loading FJC data file: education.csv
2025-07-12 18:24:41 | INFO | load_fjc_data - Loaded education data with 8040 records
2025-07-12 18:24:41 | INFO | load_fjc_csv - Loading FJC data file: federal-judicial-service.csv
2025-07-12 18:24:41 | INFO | load_fjc_data - Loaded federal_judicial_service data with 4720 records
2025-07-12 18:24:41 | INFO | load_fjc_csv - Loading FJC data file: judges.csv
2025-07-12 18:24:42 | INFO | load_fjc_data - Loaded judges data with 4022 records
2025-07-12 18:24:42 | INFO | load_fjc_csv - Loading FJC data file: other-nominations-recess.csv
2025-07-12 18:24:42 | INFO | load_fjc_data - Loaded

Loaded FJC data files:
- demographics: 4022 records
- education: 8040 records
- federal_judicial_service: 4720 records
- judges: 4022 records
- other_nominations_recess: 828 records
- other_federal_judicial_service: 611 records
- professional_career: 19003 records


### Build a "seat timeline" inferred from FJC's data about when judges were in service:

In [ ]:
from nomination_predictor.dataset import build_and_validate_seat_timeline

try:
    seat_timeline_df = build_and_validate_seat_timeline(federal_judicial_service_df)
    print(f"✅ Successfully built seat timeline with {len(seat_timeline_df):,} records")
    all_dataframes['seat_timeline'] = seat_timeline_df
except Exception as e:
    print(f"❌ Error: {e}")
    raise

2025-07-12 18:24:42 | INFO | build_seat_timeline - Building seat timeline table
2025-07-12 18:24:42 | WARNING | build_seat_timeline - Editing a derived vacancy_date: Termination date > successor commission for seat CA11SR: 1998-06-07 > 1981-10-01
2025-07-12 18:24:42 | WARNING | build_seat_timeline - Editing a derived vacancy_date: Termination date > successor commission for seat CA11SR: 1993-11-11 > 1981-10-01
2025-07-12 18:24:42 | WARNING | build_seat_timeline - Editing a derived vacancy_date: Termination date > successor commission for seat CA11SR: 2001-11-15 > 1981-10-01
2025-07-12 18:24:42 | WARNING | build_seat_timeline - Editing a derived vacancy_date: Termination date > successor commission for seat CA11SR: 1982-10-27 > 1981-10-01
2025-07-12 18:24:42 | WARNING | build_seat_timeline - Editing a derived vacancy_date: Termination date > successor commission for seat CA11SR: 1987-08-22 > 1981-10-01
2025-07-12 18:24:43 | WARNING | build_seat_timeline - Editing a derived vacancy_date:

✅ Successfully built seat timeline with 4,720 records


## 2. Congress.gov API Data

The Congress.gov API provides detailed information about judicial nominations, including:
- Nomination date
- Nominee information
- Confirmation status and date
- Committee actions

### Setup API Access

In [ ]:
# Check if API key is available
api_key = os.environ.get("CONGRESS_API_KEY")
if not api_key:
    print("❌ Error: CONGRESS_API_KEY environment variable not set")
    print("Please set the CONGRESS_API_KEY environment variable to your Congress.gov API key")
    print("You can request an API key at: https://api.congress.gov/sign-up/")
else:
    print("✓ Congress API key found in environment variables")
    # Initialize the API client
    congress_client = CongressAPIClient(api_key)
    print("✓ Congress API client initialized")

✓ Congress API key found in environment variables
✓ Congress API client initialized


### Fetch Judicial Nominations from Recent Congresses

In [ ]:
# Fetch judicial nominations from recent congresses
# Congress numbering: 116th (2019-2021), 117th (2021-2023), 118th (2023-2025)
import os
from lzma import MODE_FAST
from pathlib import Path

from nomination_predictor.config import RAW_DATA_DIR

MOST_RECENT_CONGRESS_TERM_TO_GET = 118
OLDEST_CONGRESS_TERM_TO_GET = 118#90

# Define cache file path for nominations
nominations_cache_file = os.path.join(RAW_DATA_DIR, "nominations.csv")
congresses = range(MOST_RECENT_CONGRESS_TERM_TO_GET, OLDEST_CONGRESS_TERM_TO_GET-1, -1)

# Check if we have cached data
if os.path.exists(nominations_cache_file):
    logger.info(f"Found cached nominations data at {nominations_cache_file}")
    nominations_df = pd.read_csv(nominations_cache_file, parse_dates=['receiveddate', 'authoritydate'])
    logger.info(f"Loaded {len(nominations_df)} nominations from cache; see dataframe's retrieval_date column to check when this data was fetched")
else:
    # If no cache, fetch from API
    all_nominations = []
    
    for congress in congresses:
        try:
            logger.info(f"Fetching judicial nominations for the {congress}th Congress...")
            nominations = congress_client.get_judicial_nominations(congress, auto_paginate=False) # can choose to disable auto-pagination if you want less data, but faster, for development
            logger.info(f"  ✓ Retrieved {len(nominations)} judicial nominations")
            all_nominations.extend(nominations)
        except Exception as e:
            logger.error(f"  ❌ Error fetching nominations for {congress}th Congress: {str(e)}")
    
    # Convert to DataFrame
    nominations_df = pd.DataFrame(all_nominations)
    logger.info(f"\nTotal nominations retrieved: {len(nominations_df)}")
    

2025-07-12 18:24:45 | INFO | <module> - Found cached nominations data at /home/wsl2ubuntuuser/nomination_predictor/data/raw/nominations.csv
2025-07-12 18:24:45 | INFO | <module> - Loaded 80 nominations from cache; see dataframe's retrieval_date column to check when this data was fetched


In [ ]:
# Preview the nominations
print(nominations_df.head())
all_dataframes['nominations'] = nominations_df

                                             actions authoritydate citation  \
0  {'count': 6, 'url': 'https://api.congress.gov/...    2025-05-12   PN2012   
1  {'count': 6, 'url': 'https://api.congress.gov/...    2025-05-12   PN2013   
2  {'count': 6, 'url': 'https://api.congress.gov/...    2025-03-28    PN814   
3  {'count': 11, 'url': 'https://api.congress.gov...    2025-03-28    PN771   
4  {'count': 12, 'url': 'https://api.congress.gov...    2025-03-28    PN769   

                                          committees  congress  \
0  {'count': 1, 'url': 'https://api.congress.gov/...       118   
1  {'count': 1, 'url': 'https://api.congress.gov/...       118   
2  {'count': 1, 'url': 'https://api.congress.gov/...       118   
3  {'count': 1, 'url': 'https://api.congress.gov/...       118   
4  {'count': 1, 'url': 'https://api.congress.gov/...       118   

                                         description  \
0  James Graham Lake, of the District of Columbia...   
1  Nicholas Geor

### Fetch nominees for just-retrieved nominations

In [ ]:
nominees_cache_file = os.path.join(RAW_DATA_DIR, "nominees.csv")

# Check if we have cached data
if os.path.exists(nominees_cache_file):
    print(f"Found cached nominees data at {nominees_cache_file}")
    nominees_df = pd.read_csv(nominees_cache_file)
    print(f"Loaded {len(nominees_df)} nominee records from cache")
elif 'nominee_url' not in nominations_df.columns:
    print("⚠️ No nominee_url column found in nominations_df")
else:
    print(f"Fetching nominee data for {len(nominations_df)} nominations...")

    # Filter out records without nominee_url
    valid_nominations = nominations_df[~nominations_df['nominee_url'].isna()]
    print(f"Found {len(valid_nominations)} nominations with valid nominee_url")

    # Fetch nominee data for all nominations
    nominees_data = congress_client.get_all_nominees_data(valid_nominations)

    # Convert to DataFrame
    nominees_df = pd.DataFrame(nominees_data)
    print(f"\nTotal nominees retrieved: {len(nominees_df)}")

Found cached nominees data at /home/wsl2ubuntuuser/nomination_predictor/data/raw/nominees.csv
Loaded 80 nominee records from cache


In [ ]:
# Preview the nominees
print(nominees_df.head())
all_dataframes['nominees'] = nominees_df

  firstName lastName middleName  ordinal state  congress  number  \
0     James     Lake     Graham        1    DC       118    2012   
1  Nicholas  Miranda     George        1    DC       118    2013   
2      Lisa     Wang         W.        1    DC       118     814   
3   Brandon     Long         S.        1    LA       118     771   
4     Jerry  Edwards        NaN        1    LA       118     769   

                                         nominee_url citation  nominee_id  \
0  https://api.congress.gov/v3/nomination/118/201...   PN2012  118-2012-1   
1  https://api.congress.gov/v3/nomination/118/201...   PN2013  118-2013-1   
2  https://api.congress.gov/v3/nomination/118/814...    PN814   118-814-1   
3  https://api.congress.gov/v3/nomination/118/771...    PN771   118-771-1   
4  https://api.congress.gov/v3/nomination/118/769...    PN769   118-769-1   

                data_source              retrieval_date suffix  
0  congress.gov_api_nominee  2025-07-12T16:47:35.972746    NaN 

In [ ]:
## TODO: determine whether safe to move this to other notebook, or if other code already depends on it happening this early
## Normalize column names, leaving data values as-is
#nominees_df.columns = [col.casefold().replace(' ', '_') for col in nominees_df.columns]
#print("\nNominees DataFrame columns:")
#for col in sorted(nominees_df.columns):
#     print(f"- {col}: {nominees_df[col].nunique()} unique values")

## 3. Confirm "nid" and "citation" uniqueness to later use as FJC and Congress indexes, respectively

In [54]:
# Check for uniqueness in ID fields before saving to the raw data folder
from nomination_predictor.dataset import validate_dataframe_ids

print("Checking ID uniqueness in dataframes before saving...")

uniqueness_results = validate_dataframe_ids(all_dataframes) # discovered not all dataframes treat nid as unique due to re-appointments, position changes, etc.

# Check if any dataframes have duplicate IDs
problematic_dfs = [name for name, result in uniqueness_results.items() 
                   if not result.get('is_unique', True)]

# if you want an easily-intuitive reason why a dataframe may not be able to use nid uniquely, try adding "education" to the "uniqueness required" list iterated through below, and see what it outputs.
# you'll find numerous judges who are listed multiple times for having gotten different college or university degrees over the years.
if any(name in ["judges", "demographics", "nominations", "nominees",] for name in problematic_dfs):
    logger.warning(f"⚠️ Found non-unique IDs in: {', '.join(problematic_dfs)}")
    for df_name in problematic_dfs:
        result = uniqueness_results[df_name]
        logger.warning(f"\nDuplicates in {df_name}:")
        display(result['duplicate_rows'])
else:
    
    logger.info("✓ All ID fields are unique across unique-ID-required dataframes.")

2025-07-12 18:24:46 | INFO | validate_dataframe_ids - Checking 'nid' uniqueness for dataframe 'judges'


Checking ID uniqueness in dataframes before saving...


2025-07-12 18:24:46 | INFO | check_id_uniqueness - All nid values are unique
2025-07-12 18:24:46 | INFO | validate_dataframe_ids - Checking 'nid' uniqueness for dataframe 'demographics'
2025-07-12 18:24:46 | INFO | check_id_uniqueness - All nid values are unique
2025-07-12 18:24:46 | INFO | validate_dataframe_ids - Checking 'nid' uniqueness for dataframe 'education'
2025-07-12 18:24:46 | WARNING | check_id_uniqueness - 3350 duplicate nid values found
2025-07-12 18:24:46 | WARNING | check_id_uniqueness -   1386811: appears 5 times
2025-07-12 18:24:46 | WARNING | check_id_uniqueness -   1382346: appears 5 times
2025-07-12 18:24:46 | WARNING | check_id_uniqueness -   1385901: appears 5 times
2025-07-12 18:24:46 | WARNING | check_id_uniqueness -   1377081: appears 5 times
2025-07-12 18:24:46 | WARNING | check_id_uniqueness -   1386451: appears 5 times
2025-07-12 18:24:46 | WARNING | check_id_uniqueness -   ... and 3345 more duplicate values
2025-07-12 18:24:46 | INFO | validate_dataframe_i

## 4. Save Data to Raw Directory

Save the datasets to the raw data directory for use by downstream notebooks.

In [ ]:
# Save data to the raw data directory
import os
from datetime import datetime

from nomination_predictor.config import RAW_DATA_DIR

# Create the raw data directory if it doesn't exist
os.makedirs(RAW_DATA_DIR, exist_ok=True)

# Add a timestamp for the manifest
timestamp = datetime.now().strftime("%Y%m%d")

# Save each FJC dataframe
# Save all dataframes to the raw data directory
print(f"Saving dataframes to {RAW_DATA_DIR}...")
saved_files = []

# Ensure the output directory exists
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)

# Save all dataframes from the all_dataframes collection
for name, df in all_dataframes.items():
    if df is not None and not df.empty:
        try:
            # Create filename
            output_file = RAW_DATA_DIR / f"{name}.csv"
            
            # Save to CSV
            df.to_csv(output_file, index=False)
            saved_files.append(f"{name}.csv")
            print(f"  ✓ Saved {len(df):,} records to {output_file}")
        except Exception as e:
            print(f"  ✗ Error saving {name}: {str(e)}")

# Print summary
if saved_files:
    print(f"\n✅ Successfully saved {len(saved_files)} dataframes to {RAW_DATA_DIR}")
else:
    print("\n⚠️ No dataframes were saved - check if all_dataframes is populated correctly")

# Create a manifest file to track what was saved and when
manifest_content = f"""# FJC Data Processing Manifest
Processed on: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
Note: Only column names are normalized (lowercase with underscores), data values remain unchanged
Files saved:
{chr(10).join(['- ' + file for file in saved_files])}
"""

with open(RAW_DATA_DIR / f"fjc_data_manifest_{timestamp}.txt", "w") as f:
    f.write(manifest_content)

print(f"✓ Saved {len(saved_files)} files to {RAW_DATA_DIR}")
print(f"✓ Created manifest: fjc_data_manifest_{timestamp}.txt")

Saving dataframes to /home/wsl2ubuntuuser/nomination_predictor/data/raw...


  ✓ Saved 4,022 records to /home/wsl2ubuntuuser/nomination_predictor/data/raw/judges.csv
  ✓ Saved 4,022 records to /home/wsl2ubuntuuser/nomination_predictor/data/raw/demographics.csv
  ✓ Saved 8,040 records to /home/wsl2ubuntuuser/nomination_predictor/data/raw/education.csv
  ✓ Saved 4,720 records to /home/wsl2ubuntuuser/nomination_predictor/data/raw/federal_judicial_service.csv
  ✓ Saved 828 records to /home/wsl2ubuntuuser/nomination_predictor/data/raw/other_nominations_recess.csv
  ✓ Saved 611 records to /home/wsl2ubuntuuser/nomination_predictor/data/raw/other_federal_judicial_service.csv
  ✓ Saved 19,003 records to /home/wsl2ubuntuuser/nomination_predictor/data/raw/professional_career.csv
  ✓ Saved 4,720 records to /home/wsl2ubuntuuser/nomination_predictor/data/raw/seat_timeline.csv
  ✓ Saved 80 records to /home/wsl2ubuntuuser/nomination_predictor/data/raw/nominations.csv
  ✓ Saved 80 records to /home/wsl2ubuntuuser/nomination_predictor/data/raw/nominees.csv

✅ Successfully saved 1

In [ ]:
# Save Congress API retrieved nominations to cache file
if nominations_df is not None and not nominations_df.empty:
    # Ensure directory exists
    os.makedirs(os.path.dirname(nominations_cache_file), exist_ok=True)
    print(f"Saving nominations to cache file: {nominations_cache_file}")
    nominations_df.to_csv(nominations_cache_file, index=False)
    print(f"✓ Saved {len(nominations_df)} nominations to cache")
    
if nominees_df is not None and not nominees_df.empty:
    # Ensure directory exists
    os.makedirs(os.path.dirname(nominees_cache_file), exist_ok=True)
    print(f"Saving nominees to cache file: {nominees_cache_file}")
    nominees_df.to_csv(nominees_cache_file, index=False)
    print(f"✓ Saved {len(nominees_df)} nominees to cache")

Saving nominations to cache file: /home/wsl2ubuntuuser/nomination_predictor/data/raw/nominations.csv
✓ Saved 80 nominations to cache
Saving nominees to cache file: /home/wsl2ubuntuuser/nomination_predictor/data/raw/nominees.csv
✓ Saved 80 nominees to cache


## Summary

In this notebook, we have:

1. Loaded Federal Judicial Center (FJC) data, the canonical source for judicial seats and judges
2. Built the seat timeline dataframe
3. Fetched judicial nominations from the Congress.gov API
4. Saved all datasets to the raw data directory for further processing by downstream notebooks

The next notebook (e.g. 1.##-nw-feature-engineering.ipynb) will load these datasets, clean them, and engineer features for modeling.